--- Day 5: Supply Stacks ---

The expedition can depart as soon as the final supplies have been unloaded from the ships. Supplies are stored in stacks of marked crates, but because the needed supplies are buried under many other crates, the crates need to be rearranged.

The ship has a giant cargo crane capable of moving crates between stacks. To ensure none of the crates get crushed or fall over, the crane operator will rearrange them in a series of carefully-planned steps. After the crates are rearranged, the desired crates will be at the top of each stack.

The Elves don't want to interrupt the crane operator during this delicate procedure, but they forgot to ask her which crate will end up where, and they want to be ready to unload them as soon as possible so they can embark.

They do, however, have a drawing of the starting stacks of crates and the rearrangement procedure (your puzzle input). For example:

    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2

In this example, there are three stacks of crates. Stack 1 contains two crates: crate Z is on the bottom, and crate N is on top. Stack 2 contains three crates; from bottom to top, they are crates M, C, and D. Finally, stack 3 contains a single crate, P.

Then, the rearrangement procedure is given. In each step of the procedure, a quantity of crates is moved from one stack to a different stack. In the first step of the above rearrangement procedure, one crate is moved from stack 2 to stack 1, resulting in this configuration:

[D]        
[N] [C]    
[Z] [M] [P]
 1   2   3 

In the second step, three crates are moved from stack 1 to stack 3. Crates are moved one at a time, so the first crate to be moved (D) ends up below the second and third crates:

        [Z]
        [N]
    [C] [D]
    [M] [P]
 1   2   3

Then, both crates are moved from stack 2 to stack 1. Again, because crates are moved one at a time, crate C ends up below crate M:

        [Z]
        [N]
[M]     [D]
[C]     [P]
 1   2   3

Finally, one crate is moved from stack 1 to stack 2:

        [Z]
        [N]
        [D]
[C] [M] [P]
 1   2   3

The Elves just need to know which crate will end up on top of each stack; in this example, the top crates are C in stack 1, M in stack 2, and Z in stack 3, so you should combine these together and give the Elves the message CMZ.

After the rearrangement procedure completes, what crate ends up on top of each stack?


In [98]:
import copy

In [99]:
def open_file():
    """open the input file"""
    with open('chris_davis_input_day5.txt') as f:
        input_data = f.readlines()
    return input_data

In [100]:
def split_input(input_data):
    """splits input into stack and moves"""
    stack_list = []
    move_list = []
    
    moves_flag = 0
    for row in input_data:
        stripped_row = row.strip()
        if moves_flag == 0:
            if stripped_row == '':
                moves_flag = 1
            else:
                stack_list.append(stripped_row)
        elif moves_flag == 1:
            move_list.append(stripped_row)
    return stack_list, move_list    

In [101]:
def convert_stacks_to_dict(stack_list):
    """
    converts the stack string to a list (for each row) of a dictionary of which item is in each stack at that row.
    Also returns the total number of stacks
    """
    
    num_stacks = int(stack_list[-1].split()[-1])
    
    list_of_stack_dicts = []
    for row in stack_list[:-1]:
        list_of_stack_dicts.append(split_stack_string(num_stacks, row))
    return list_of_stack_dicts, num_stacks

In [102]:
def split_stack_string(num_stacks, stack_string):
    """returns a dictionary of non-empty items in the stack"""
    stack_dict = {}
    for stack_num in range(num_stacks):
        item_in_stack = stack_string[4 * stack_num + 1]
        if item_in_stack.strip() != '':
            stack_dict[stack_num] = item_in_stack
    return stack_dict

In [103]:
def create_stacks(list_of_stack_dicts, num_stacks):
    """
    Turns a list of dictionaries for each row-stack into a single dictionary with a list of items in each stack, 
    ordered such that the first item in the list is the top item
    """
    stack_dict = {}
    for stack_num in range(0, num_stacks):
        stack_list = []
        for stack_row in list_of_stack_dicts:
            if stack_row.get(stack_num):
                stack_list.append(stack_row.get(stack_num))
        stack_dict[stack_num] = stack_list.copy()
    return stack_dict

In [117]:
def move_one(from_stack, to_stack, stack_dict):
    """moves the top item from one stack to another"""
    new_stack_dict = copy.deepcopy(stack_dict)
    moved_item = new_stack_dict[from_stack].pop(0)
    new_stack_dict[to_stack].insert(0, moved_item)
    return new_stack_dict

def move_many(from_stack, to_stack, stack_dict, num_moves):
    """moves many items from one stack to another"""
    new_stack_dict = move_one(from_stack, to_stack, stack_dict)
    if num_moves > 1:
        new_stack_dict = move_many(from_stack, to_stack, new_stack_dict, num_moves - 1)
    return new_stack_dict

In [175]:
def parse_moves(move_list):
    """parse the move text to a useful dict. Because we use Python here, decrement all column names by 1 (ex: 0->8 instead of 1->9)"""
    moves = []
    for moveset in move_list:
        parsed = moveset.split(' ')
        moves.append({'num_moves': int(parsed[1]), 'from_stack': int(parsed[3]) - 1, 'to_stack': int(parsed[5]) - 1})
    return moves

In [176]:
def execute_all_moves(moves, stack_dict):
    """run all the moves in order"""
    updated_stack_dict = copy.deepcopy(stack_dict)
    for move in moves:
        updated_stack_dict = move_many(move['from_stack'], move['to_stack'], updated_stack_dict, move['num_moves'])
    return updated_stack_dict

In [189]:
def get_top_of_stacks(stack_dict, num_stacks):
    """concatenate all the strings of each item in the stack dict, in order"""
    output_str = ""
    for i in range(num_stacks):
        output_str = output_str + stack_dict[i][0]
    return output_str

In [177]:
input_data = open_file()

In [178]:
stack_list, move_list = split_input(input_data)

In [179]:
list_of_stack_dicts, num_stacks = convert_stacks_to_dict(stack_list)

In [180]:
stack_dict = create_stacks(list_of_stack_dicts, num_stacks)

In [181]:
moves = parse_moves(move_list)

In [186]:
final_stack = execute_all_moves(moves, stack_dict)

In [190]:
get_top_of_stacks(final_stack, num_stacks)

'SBPQRSCDF'

--- Part Two ---

As you watch the crane operator expertly rearrange the crates, you notice the process isn't following your prediction.

Some mud was covering the writing on the side of the crane, and you quickly wipe it away. The crane isn't a CrateMover 9000 - it's a CrateMover 9001.

The CrateMover 9001 is notable for many new and exciting features: air conditioning, leather seats, an extra cup holder, and the ability to pick up and move multiple crates at once.

Again considering the example above, the crates begin in the same configuration:

    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 

Moving a single crate from stack 2 to stack 1 behaves the same as before:

[D]        
[N] [C]    
[Z] [M] [P]
 1   2   3 

However, the action of moving three crates from stack 1 to stack 3 means that those three moved crates stay in the same order, resulting in this new configuration:

        [D]
        [N]
    [C] [Z]
    [M] [P]
 1   2   3

Next, as both crates are moved from stack 2 to stack 1, they retain their order as well:

        [D]
        [N]
[C]     [Z]
[M]     [P]
 1   2   3

Finally, a single crate is still moved from stack 1 to stack 2, but now it's crate C that gets moved:

        [D]
        [N]
        [Z]
[M] [C] [P]
 1   2   3

In this example, the CrateMover 9001 has put the crates in a totally different order: MCD.

Before the rearrangement process finishes, update your simulation so that the Elves know where they should stand to be ready to unload the final supplies. After the rearrangement procedure completes, what crate ends up on top of each stack?


In [191]:
def multi_move(from_stack, to_stack, stack_dict, num_moves):
    """moves the top {num_moves} items from one stack to another"""
    new_stack_dict = copy.deepcopy(stack_dict)
    moved_items = new_stack_dict[from_stack][:num_moves]
    
    # update from stack
    new_stack_dict[from_stack] = new_stack_dict[from_stack][num_moves:]
    new_stack_dict[to_stack] = moved_items + new_stack_dict[to_stack]
    return new_stack_dict

In [195]:
def execute_all_part_2_moves(moves, stack_dict):
    """run all the moves in order"""
    updated_stack_dict = copy.deepcopy(stack_dict)
    for move in moves:
        updated_stack_dict = multi_move(move['from_stack'], move['to_stack'], updated_stack_dict, move['num_moves'])
    return updated_stack_dict

In [196]:
execute_all_part_2_moves(moves, stack_dict)

{0: ['R', 'J', 'P', 'C', 'V', 'D', 'Z', 'S', 'T', 'D', 'M', 'R', 'F'],
 1: ['G', 'B', 'M'],
 2: ['L', 'V', 'D', 'N', 'B'],
 3: ['V', 'C', 'N', 'F'],
 4: ['R', 'Q', 'G', 'S', 'M', 'D', 'L'],
 5: ['C', 'H', 'H', 'T', 'R', 'C'],
 6: ['Q', 'P', 'V', 'J', 'S', 'L', 'S', 'W'],
 7: ['S', 'F', 'N', 'Q', 'J', 'P', 'R', 'P', 'S'],
 8: ['B']}

In [192]:
stack_dict

{0: ['V', 'N', 'F', 'S', 'M', 'P', 'H', 'J'],
 1: ['Q', 'D', 'J', 'M', 'L', 'R', 'S'],
 2: ['B', 'W', 'S', 'C', 'H', 'D', 'Q', 'N'],
 3: ['L', 'C', 'S', 'R'],
 4: ['B', 'F', 'P', 'T', 'V', 'M'],
 5: ['C', 'N', 'Q', 'R', 'T'],
 6: ['R', 'V', 'G'],
 7: ['R', 'L', 'D', 'P', 'S', 'Z', 'C'],
 8: ['F', 'B', 'P', 'G', 'V', 'J', 'S', 'D']}